In [1]:
import os
import time
import string
from collections import defaultdict


In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from utils import timer

In [3]:
tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [17]:
puncts=['☹', 'Ź', 'Ż', 'ἰ', 'ή', 'Š', '＞', 'ξ','ฉ', 'ั', 'น', 'จ', 'ะ', 'ท', 'ำ', 'ใ', 'ห', '้', 'ด', 'ี', 
'่', 'ส', 'ุ', 'Π', 'प', 'ऊ', 'Ö', 'خ', 'ب', 'ஜ', 'ோ', 'ட', '', '「', 'ẽ', '½', '△', 'É', 'ķ', 'ï', '¿', 
'ł', '북', '한', '¼', '∆', '≥', '⇒', '¬', '∨', 'č', 'š', '∫', 'ḥ', 'ā', 'ī', 'Ñ', 'à', '▾', 'Ω', '＾', 'ý', 
'µ', '?', '!', '.', ',', '"', '#', '$', '%', '\\', "'", '(', ')', '*', '+', '-', '/', ':', ';', '<', '=', 
'>', '@', '[', ']', '^', '_', '`', '{', '|', '}', '~', '“', '”', '’', 'é', 'á', '′', '…', 'ɾ', '̃', 
'ɖ', 'ö', '–', '‘', 'ऋ', 'ॠ', 'ऌ', 'ॡ', 'ò', 'è', 'ù', 'â', 'ğ', 'म', 'ि', 'ल', 'ग', 'ई', 'क', 'े', 'ज', 
'ो', 'ठ', 'ं', 'ड', 'Ž', 'ž', 'ó', '®', 'ê', 'ạ', 'ệ', '°', 'ص', 'و', 'ر', 'ü', '²', '₹', 'ú', '√', 'α', 
'→', 'ū', '—', '£', 'ä', '️', 'ø', '´', '×', 'í', 'ō', 'π', '÷', 'ʿ', '€', 'ñ', 'ç', 'へ', 'の', 'と', 'も',
'↑', '∞', 'ʻ', '℅''ι', '•', 'ì', '−', 'л', 'я', 'д', 'ل', 'ك', 'م', 'ق', 'ا', '∈', '∩', '⊆', 'ã', 'अ', 'न',
'ु', 'स', '्', 'व', 'ा', 'र', 'त', '§', '℃', 'θ', '±', '≤', 'उ', 'द', 'य', 'ब', 'ट', '͡', '͜', 'ʖ', '⁴', 
'™', 'ć', 'ô', 'с', 'п', 'и', 'б', 'о', 'г', '≠', '∂', 'आ', 'ह', 'भ', 'ी', '³', 'च', '...', '⌚', '⟨', '⟩',
'∖', '˂', 'ⁿ', '⅔', 'న', 'ీ', 'క', 'ె', 'ం', 'ద', 'ు', 'ా', 'గ', 'ర', 'ి', 'చ', 'র', 'ড়', 'ঢ়', 'સ',
'ં', 'ઘ', 'ર', 'ા', 'જ', '્', 'ય', 'ε', 'ν', 'τ', 'σ', 'ş', 'ś', 'س', 'ت', 'ط', 'ي', 'ع', 'ة', 'د', 'Å',
'☺', 'ℇ', '❤', '♨', '✌', 'ﬁ', 'て', '„', 'Ā', 'ត', 'ើ', 'ប', 'ង', '្', 'អ', 'ូ', 'ន', 'ម', 'ា', 'ធ', 'យ',
'វ', 'ី', 'ខ', 'ល', 'ះ', 'ដ', 'រ', 'ក', 'ឃ', 'ញ', 'ឯ', 'ស', 'ំ', 'ព', 'ិ', 'ៃ', 'ទ', 'គ', '¢', 'つ', 'や', 
'ค', 'ณ', 'ก', 'ล', 'ง', 'อ', 'ไ', 'ร', 'į', 'ی', 'ю', 'ʌ', 'ʊ', 'י', 'ה', 'ו', 'ד', 'ת', 'ᠠ', 'ᡳ', 'ᠰ', 
'ᠨ', 'ᡤ', 'ᡠ', 'ᡵ', 'ṭ', 'ế', 'ध', 'ड़', 'ß', '¸', 'ч',  'ễ', 'ộ', 'फ', 'μ', '⧼', '⧽', 'ম', 'হ', 'া', 'ব', 
'ি', 'শ', '্', 'প', 'ত', 'ন', 'য়', 'স', 'চ', 'ছ', 'ে', 'ষ', 'য', '়', 'ট', 'উ', 'থ', 'ক', 'ῥ', 'ζ', 'ὤ', 'Ü', 
'Δ', '내', '제', 'ʃ', 'ɸ', 'ợ', 'ĺ', 'º', 'ष', '♭', '़', '✅', '✓', 'ě', '∘', '¨', '″', 'İ', '⃗', '̂', 'æ', 'ɔ', 
'∑', '¾', 'Я', 'х', 'О', 'з', 'ف', 'ن', 'ḵ', 'Č', 'П', 'ь', 'В', 'Φ', 'ỵ', 'ɦ', 'ʏ', 'ɨ', 'ɛ', 'ʀ', 'ċ', 'օ', 
'ʍ', 'ռ', 'ք', 'ʋ', '兰', 'ϵ', 'δ', 'Ľ', 'ɒ', 'î', 'Ἀ', 'χ', 'ῆ', 'ύ', 'ኤ', 'ል', 'ሮ', 'ኢ', 'የ', 'ኝ', 'ን', 
'አ', 'ሁ', '≅', 'ϕ', '‑', 'ả', '￼', 'ֿ', 'か', 'く', 'れ', 'ő', '－', 'ș', 'ן', 'Γ', '∪', 'φ', 'ψ', '⊨', 'β', '∠', 
'Ó', '«', '»', 'Í', 'க', 'வ', 'ா', 'ம', '≈', '⁰', '⁷', 'ấ', 'ũ', '눈', '치', 'ụ', 'å', '،', '＝', '（', '）', 
'ə', 'ਨ', 'ਾ', 'ਮ', 'ੁ', '︠', '︡', 'ɑ', 'ː', 'λ', '∧', '∀', 'Ō', 'ㅜ', 'Ο', 'ς', 'ο', 'η', 'Σ', 'ण', '大','能', 
'化', '生', '水', '谷', '精', '微', 'ル', 'ー', 'ジ', 'ュ', '支', '那', '¹', 'マ', 'リ', '仲', '直', 'り', 'し', 'た', 
'主', '席', '血', '⅓', '漢', '髪', '金', '茶', '訓', '読', '黒', 'ř', 'あ', 'わ', 'る', '胡', '南', '수', '능', '广', 
'电', '总', 'ί', '서', '로', '가', '를', '행', '복', '하', '게', '기', '乡', '故', '爾', '汝', '言', '得', '理', '让', 
'骂', '野', '比', 'び', '太', '後', '宮', '甄', '嬛', '傳', '做', '莫', '你', '酱', '紫', '甲', '骨', '陳', '宗', '陈', 
'什', '么', '说', '伊', '藤', '長', 'ﷺ', '僕', 'だ', 'け', 'が', '街', '◦', '火', '团', '表',  '看', '他', '顺', '眼', 
'中', '華', '民', '國', '許', '自', '東', '儿', '臣', '惶', '恐', 'っ', '木', 'ホ', 'ج', '教', '官', '국', '고', '등', 
'학', '교', '는', '몇', '시', '간', '업', '니', '本', '語', '上', '手', 'で', 'ね', '台', '湾', '最', '美', '风', '景', 
'Î', '≡', '皎', '滢', '杨', '∛', '簡', '訊', '短', '送', '發', 'お', '早', 'う', '朝', 'ش', 'ه', '饭', '乱', '吃', 
'话', '讲', '男', '女', '授', '受', '亲', '好', '心', '没', '报', '攻', '克', '禮', '儀', '統', '已', '經', '失', '存', 
'٨', '八', '‛', '字', '：', '别', '高', '兴', '还', '几', '个', '条', '件', '呢', '觀', '《', '》', '記', '宋', '楚', 
'瑜', '孫', '瀛', '枚', '无', '挑', '剔', '聖', '部', '頭', '合', '約', 'ρ', '油', '腻', '邋', '遢', 'ٌ', 'Ä', '射', '籍', 
'贯', '老', '常', '谈', '族', '伟', '复', '平', '天', '下', '悠', '堵', '阻', '愛', '过', '会', '俄', '罗', '斯', '茹', 
'西', '亚', '싱', '관', '없', '어', '나', '이', '키', '夢', '彩', '蛋', '鰹', '節', '狐', '狸', '鳳', '凰', '露', '王', 
'晓', '菲', '恋', 'に', '落', 'ち', 'ら', 'よ', '悲', '反', '清', '復', '明', '肉', '希', '望', '沒', '公', '病', '配', 
'信', '開', '始', '日', '商', '品', '発', '売', '分', '子', '创', '意', '梦', '工', '坊', 'ک', 'پ', 'ڤ', '蘭', '花', '羡', 
'慕', '和', '嫉', '妒', '是', '样', 'ご', 'め', 'な', 'さ', 'い', 'す', 'み', 'ま', 'せ', 'ん', '音', '红', '宝', '书', 
'封', '柏', '荣', '江', '青', '鸡', '汤', '文', '粵', '拼', '寧', '可', '錯', '殺', '千', '絕', '放', '過', '」', '之', 
'勢', '请', '国', '知', '识', '产', '权', '局', '標', '點', '符', '號', '新', '年', '快', '乐', '学', '业', '进', '步', '身', 
'体', '健', '康', '们', '读', '我', '的', '翻', '译', '篇', '章', '欢', '迎', '入', '坑', '有', '毒', '黎', '氏', '玉', '英', 
'啧', '您', '这', '口', '味', '奇', '特', '也', '就', '罢', '了', '非', '要', '以', '此', '为', '依', '据', '对', '人', '家', 
'批', '判', '一', '番', '不', '地', '道', '啊', '谢', '六', '佬']
punctString = ''.join([p for p in puncts])

In [18]:
TRAIN_PATH = "train/train.csv"

In [19]:
def load_data(file_path):
	return pd.read_csv(file_path)

In [20]:
def remove_punctuations(text):
   #return ' '.join([t for t in text.split(' ') if t not in puncts])
    #for t in text:
     #   if t in puncts:
      #      text = text.replace(t, "")
    #return text
    return text.translate(str.maketrans('','',punctString))

In [21]:
print(remove_punctuations('ab#//#adfaacd# adf #/a'))
print(remove_punctuations('Hello, my name is Bruh!'))

abadfaacd adf a
Hello my name is Bruh


In [22]:
def init_tokenizer():
    return RegexpTokenizer('\w+')

In [23]:
stopwordsList = stopwords.words('english')

In [24]:
print (stopwordsList[:15])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours']


In [25]:
def remove_stopwords(text):
    return ' '.join([word for word in text.split(' ') if word not in stopwordsList])

In [26]:
def init_stemmer():
	return SnowballStemmer('english')

In [27]:
def clean_data(data, punctuation=True, tokenize=True, removeStopwords=True, stemming=True):
	if punctuation:
		print('Running remove_punctuations...')
		data['question_text'] = data['question_text'].progress_apply(lambda x: remove_punctuations(x))
		print(data['question_text'].head(15))

	if tokenize:
		print('Running tokenize...')
		data['question_text'] = data['question_text'].progress_apply(lambda x: ' '.join(init_tokenizer().tokenize(x.lower())))
		print(data['question_text'].head(15))

	if removeStopwords:
		
		print('Removing stopwords...')
		data['question_text'] = data['question_text'].progress_apply(lambda x: remove_stopwords(x))
		print(data['question_text'].head(15))

	if stemming:
		print('Stemming...')
		stemmer = init_stemmer()
		data['question_text'] = data['question_text'].progress_apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split(' ')]))
		print(data['question_text'].head(15))
        

In [28]:
def load_clean_data(data_path = TRAIN_PATH, output_file = "train/train1111.csv", punctuation = True, tokenize = True, remove_stopwords = True, stemming = True):
    data = load_data(data_path)
    clean_data(data, punctuation, tokenize, remove_stopwords, stemming)
    if output_file :
        data.to_csv(output_file, 
					   index=False)
    return data

In [29]:
data = load_clean_data(data_path = TRAIN_PATH, output_file = "train/train1101.csv", punctuation = True, tokenize = True, remove_stopwords = False, stemming = True)

  0%|                                                          | 722/1306122 [00:00<03:18, 6580.62it/s]

Running remove_punctuations...


  0%|▎                                                       | 5983/1306122 [00:00<00:22, 57242.20it/s]

0     How did Quebec nationalists see their province...
1     Do you have an adopted dog how would you encou...
2     Why does velocity affect time Does velocity af...
3     How did Otto von Guericke used the Magdeburg h...
4     Can I convert montra helicon D to a mountain b...
5     Is Gaza slowly becoming Auschwitz Dachau or Tr...
6     Why does Quora automatically ban conservative ...
7     Is it crazy if I wash or wipe my groceries off...
8     Is there such a thing as dressing moderately a...
9     Is it just me or have you ever been in this ph...
10                      What can you say about feminism
11                  How were the Calgary Flames founded
12    What is the dumbest yet possibly true explanat...
13    Can we use our external hard disk as a OS as w...
14    I am 30 living at home and have no boyfriend I...
Name: question_text, dtype: object
Running tokenize...


  0%|                                                          | 297/1306122 [00:00<07:22, 2948.67it/s]

0     how did quebec nationalists see their province...
1     do you have an adopted dog how would you encou...
2     why does velocity affect time does velocity af...
3     how did otto von guericke used the magdeburg h...
4     can i convert montra helicon d to a mountain b...
5     is gaza slowly becoming auschwitz dachau or tr...
6     why does quora automatically ban conservative ...
7     is it crazy if i wash or wipe my groceries off...
8     is there such a thing as dressing moderately a...
9     is it just me or have you ever been in this ph...
10                      what can you say about feminism
11                  how were the calgary flames founded
12    what is the dumbest yet possibly true explanat...
13    can we use our external hard disk as a os as w...
14    i am 30 living at home and have no boyfriend i...
Name: question_text, dtype: object
Stemming...


100%|██████████████████████████████████████████████████████| 1306122/1306122 [05:19<00:00, 4087.81it/s]


0     how did quebec nationalist see their provinc a...
1     do you have an adopt dog how would you encoura...
2     whi doe veloc affect time doe veloc affect spa...
3     how did otto von guerick use the magdeburg hem...
4     can i convert montra helicon d to a mountain b...
5     is gaza slowli becom auschwitz dachau or trebl...
6     whi doe quora automat ban conserv opinion when...
7     is it crazi if i wash or wipe my groceri off g...
8     is there such a thing as dress moder and if so...
9     is it just me or have you ever been in this ph...
10                         what can you say about femin
11                     how were the calgari flame found
12    what is the dumbest yet possibl true explan fo...
13    can we use our extern hard disk as a os as wel...
14    i am 30 live at home and have no boyfriend i w...
Name: question_text, dtype: object
